In [15]:
! pip3 install pyecharts

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
from pyecharts import options as opts
from pyecharts.charts import PictorialBar, Line, Bar
from pyecharts.globals import SymbolType
from pyecharts.charts import Geo
from pyecharts.globals import ChartType
from pyecharts.commons.utils import JsCode
import pandas as pd
import re

In [31]:
provincial = pd.read_csv('provincial_capital.csv')  # 读取省会信息
provincial

,provincial,city
0,北京,北京
1,天津,天津
2,上海,上海
3,重庆,重庆
4,黑龙江,哈尔滨
5,吉林,长春
6,辽宁,沈阳
7,河北,石家庄
8,内蒙古,呼和浩特
9,河南,郑州


In [32]:
china_city_code = pd.read_csv('china-city-list.csv')  # 读取城市 id 信息
china_city_code

,City_ID,City_EN,City_CN,Country_code,Country_EN,Country_CN,Province_EN,Province_CN,Admin_ district_EN,Admin_ district_CN,Latitude,Longitude,AD_code,Unnamed: 13
0,CN101010100,beijing,北京,CN,China,中国,beijing,北京,beijing,北京,39.904987,116.40529,"110100,110000,100000",NaN
1,CN101010200,haidian,海淀,CN,China,中国,beijing,北京,beijing,北京,39.956074,116.31032,110108,NaN
2,CN101010300,chaoyang,朝阳,CN,China,中国,beijing,北京,beijing,北京,39.921490,116.48641,110105,NaN
3,CN101010400,shunyi,顺义,CN,China,中国,beijing,北京,beijing,北京,40.128937,116.65353,110113,NaN
4,CN101010500,huairou,怀柔,CN,China,中国,beijing,北京,beijing,北京,40.324272,116.63712,110116,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,CN101340402,miaoli,苗栗,CN,China,中国,taiwan,台湾,taizhong,台中,24.558000,120.81200,710000,NaN
3236,CN101340403,zhanghua,彰化,CN,China,中国,taiwan,台湾,taizhong,台中,24.077000,120.53500,710000,NaN
3237,CN101340404,nantou,南投,CN,China,中国,taiwan,台湾,taizhong,台中,23.916000,120.68500,710000,NaN
3238,CN101340405,hualian,花莲,CN,China,中国,taiwan,台湾,taizhong,台中,23.983000,121.60300,710000,NaN


In [34]:
weather = pd.read_csv("weather_data.csv")  # 读取天气信息
weather

,city,time,wea,tem,wind,wind_level
0,兰州,周四（12日）,多云转雨,25℃/14℃,北风转东南风,3-4级转<3级
1,兰州,周五（13日）,雨,18℃/12℃,东北风,<3级
2,兰州,周六（14日）,雨,17℃/12℃,东北风转北风,<3级
3,兰州,周日（15日）,雨转晴,21℃/13℃,东南风,<3级
4,兰州,周一（16日）,雨,24℃/15℃,东南风,<3级
...,...,...,...,...,...,...
267,北京,周日（15日）,雨转多云,27℃/13℃,南风转东风,<3级
268,北京,周一（16日）,雨转阴,29℃/14℃,南风转东北风,<3级
269,北京,周二（17日）,雨转多云,27℃/13℃,东南风,<3级
270,北京,周三（18日）,多云转雨,27℃/16℃,南风,<3级


In [35]:
# 查看天气总体情况
weather['wea'].value_counts()

雨        112
雨转阴       22
雨转多云      20
多云        18
多云转雨      15
多云转阴      15
阴         11
多云转晴      10
晴          9
雨转晴        8
阴转雨        7
小雨转雨       6
阴转多云       6
晴转阴        3
晴转多云       3
雷阵雨转雨      2
小雨转晴       2
晴转雨        2
阵雨转雨       1
Name: wea, dtype: int64

In [36]:
# 转换天气变量，数值越多，说明降水概率越大
weather_dict = {
    "snow": 100,
    "rain": 80,
    "cloud": 50,
    "overcast": 60,
    "sun": 20
}

In [37]:
# 提取省会城市 id
provincial_data = pd.DataFrame()
for i in provincial['city'].values.tolist():
    for j in china_city_code['City_CN'].values.tolist():
        if j == i:
            provincial_data = pd.concat([china_city_code[china_city_code['City_CN'] == j], provincial_data])
provincial_data

,City_ID,City_EN,City_CN,Country_code,Country_EN,Country_CN,Province_EN,Province_CN,Admin_ district_EN,Admin_ district_CN,Latitude,Longitude,AD_code,Unnamed: 13
1394,CN101160101,lanzhou,兰州,CN,China,中国,gansu,甘肃,lanzhou,兰州,36.058040,103.823555,"620101,620100,620000",NaN
3188,CN101310101,haikou,海口,CN,China,中国,hainan,海南,haikou,海口,20.031971,110.331190,"460101,460100,460000",NaN
3225,CN101340101,Taibei,台北,CN,China,中国,taiwan,台湾,taibei,台北,25.040000,121.516000,710000,NaN
3222,CN101330101,macao,澳门,CN,China,中国,macao,澳门,macao,澳门,22.202000,113.544000,"820000,820001,820002,820003,820004,820005,820007",NaN
3219,CN101320101,hongkong,香港,CN,China,中国,hongkong,香港,hongkong,香港,22.307000,114.177000,"810000,810001,810002,810003,810004,810005,8100...",NaN
2138,CN101230101,fuzhou,福州,CN,China,中国,fujian,福建,fuzhou,福州,26.075302,119.306240,"350101,350100,350000",NaN
1915,CN101210101,hangzhou,杭州,CN,China,中国,zhejiang,浙江,hangzhou,杭州,30.287458,120.153580,"330101,330100,330000",NaN
1491,CN101170101,yinchuan,银川,CN,China,中国,ningxia,宁夏,yinchuan,银川,38.466370,106.278175,"640101,640100,640000",NaN
736,CN101100101,taiyuan,太原,CN,China,中国,shanxi,山西,taiyuan,太原,37.857014,112.549250,"140101,140100,140000",NaN
980,CN101120101,jinan,济南,CN,China,中国,shandong,山东,jinan,济南,36.675808,117.000920,"370101,370100,370000",NaN


In [38]:
# 按照城市分组
wea_group = weather.groupby('city').apply(lambda x: x[:])
wea_group.head(50)

city     time   wea      tem     wind wind_level
city                                                      
上海   248    上海  周四（12日）  多云转晴  30℃/23℃   北风转西北风        <3级
     249    上海  周五（13日）  雨转多云  31℃/24℃   东风转东北风   3-4级转<3级
     250    上海  周六（14日）  雨转多云  31℃/25℃   东风转东北风   3-4级转<3级
     251    上海  周日（15日）     雨  30℃/25℃      东北风       3-4级
     252    上海  周一（16日）     雨  29℃/25℃      东北风       3-4级
     253    上海  周二（17日）     雨  28℃/23℃      东北风       3-4级
     254    上海  周三（18日）    多云  27℃/22℃   东北风转东风       4-5级
     255    上海  周四（19日）     雨  28℃/24℃    东风转北风       3-4级
乌鲁木齐 176  乌鲁木齐  周四（12日）  小雨转晴   17℃/9℃   西北风转南风   4-5级转<3级
     177  乌鲁木齐  周五（13日）  阴转多云   20℃/9℃    北风转南风        <3级
     178  乌鲁木齐  周六（14日）     阴   20℃/9℃       北风        <3级
     179  乌鲁木齐  周日（15日）  多云转阴  23℃/12℃   东北风转西风        <3级
     180  乌鲁木齐  周一（16日）  多云转阴  21℃/12℃   西北风转西风        <3级
     181  乌鲁木齐  周二（17日）  多云转阴  20℃/12℃   西北风转西风        <3级
     182  乌鲁木齐  周三（18日）     雨  20℃/12℃  东北风转西南风        <3级
     183  乌鲁木齐  周四（19日）    多云  19℃/11℃  东北风转西南风        <3级
兰州   0      兰州  周四（12日）  多云转雨  25℃/14℃   北风转东南风   3-4级转<3级
     1      兰州  周五（13日）     雨  18℃/12℃      东北风        <3级
     2      兰州  周六（14日）     雨  17℃/12℃   东北风转北风        <3级
     3      兰州  周日（15日）   雨转晴  21℃/13℃      东南风        <3级
     4      兰州  周一（16日）     雨  24℃/15℃      东南风        <3级
     5      兰州  周二（17日）     雨  25℃/15℃      东北风        <3级
     6      兰州  周三（18日）     雨  17℃/10℃      东北风        <3级
     7      兰州  周四（19日）   雨转阴  17℃/10℃   北风转东北风        <3级
北京   264    北京  周四（12日）  多云转阴  27℃/16℃   东北风转西风        <3级
     265    北京  周五（13日）     雨  29℃/19℃   西南风转北风        <3级
     266    北京  周六（14日）     阴  28℃/16℃    东风转北风        <3级
     267    北京  周日（15日）  雨转多云  27℃/13℃    南风转东风        <3级
     268    北京  周一（16日）   雨转阴  29℃/14℃   南风转东北风        <3级
     269    北京  周二（17日）  雨转多云  27℃/13℃      东南风        <3级
     270    北京  周三（18日）  多云转雨  27℃/16℃       南风        <3级
     271    北京  周四（19日）   雨转阴  29℃/13℃   南风转东北风   <3级转3-4级
南京   80     南京  周四（12日）     晴  27℃/21℃   东风转东北风        <3级
     81     南京  周五（13日）    多云  31℃/23℃   东北风转东风        <3级
     82     南京  周六（14日）  多云转晴  32℃/23℃   东风转东北风   3-4级转<3级
     83     南京  周日（15日）  雨转多云  32℃/24℃      东北风       3-4级
     84     南京  周一（16日）  雨转多云  30℃/21℃      东北风   3-4级转<3级
     85     南京  周二（17日）  多云转晴  30℃/20℃   东风转东北风       3-4级
     86     南京  周三（18日）    多云  29℃/20℃       东风   3-4级转<3级
     87     南京  周四（19日）  雨转多云  30℃/22℃       东风       3-4级
南宁   128    南宁  周四（12日）  小雨转雨  31℃/22℃    南风转北风        <3级
     129    南宁  周五（13日）     雨  31℃/23℃   北风转西北风        <3级
     130    南宁  周六（14日）  雨转多云  32℃/24℃   北风转西北风        <3级
     131    南宁  周日（15日）     雨  34℃/24℃   北风转西北风        <3级
     132    南宁  周一（16日）     雨  31℃/23℃   南风转东南风        <3级
     133    南宁  周二（17日）     雨  25℃/22℃   东风转西北风        <3级
     134    南宁  周三（18日）     雨  28℃/22℃      西北风        <3级
     135    南宁  周四（19日）     雨  31℃/24℃   北风转西北风        <3级
南昌   96     南昌  周四（12日）  多云转晴  34℃/25℃   北风转东北风   3-4级转<3级
     97     南昌  周五（13日）     晴  35℃/25℃   东北风转北风        <3级

In [39]:
# 提取中秋节当天天气信息
twoone = wea_group[wea_group['time'] == '周五（13日）']
twoone

,,city,time,wea,tem,wind,wind_level
city,,,,,,,
上海,249,上海,周五（13日）,雨转多云,31℃/24℃,东风转东北风,3-4级转<3级
乌鲁木齐,177,乌鲁木齐,周五（13日）,阴转多云,20℃/9℃,北风转南风,<3级
兰州,1,兰州,周五（13日）,雨,18℃/12℃,东北风,<3级
北京,265,北京,周五（13日）,雨,29℃/19℃,西南风转北风,<3级
南京,81,南京,周五（13日）,多云,31℃/23℃,东北风转东风,<3级
南宁,129,南宁,周五（13日）,雨,31℃/23℃,北风转西北风,<3级
南昌,97,南昌,周五（13日）,晴,35℃/25℃,东北风转北风,<3级
台北,17,台北,周五（13日）,雨,29℃/26℃,东北风转东风,4-5级
合肥,89,合肥,周五（13日）,多云转晴,31℃/22℃,东风,<3级


In [40]:
# 城市信息
twoone_ctiy = twoone['city'].values.tolist()
twoone_ctiy = list(twoone_ctiy)
twoone_ctiy

['上海',
 '乌鲁木齐',
 '兰州',
 '北京',
 '南京',
 '南宁',
 '南昌',
 '台北',
 '合肥',
 '呼和浩特',
 '哈尔滨',
 '天津',
 '太原',
 '广州',
 '成都',
 '拉萨',
 '昆明',
 '杭州',
 '武汉',
 '沈阳',
 '济南',
 '海口',
 '澳门',
 '石家庄',
 '福州',
 '西宁',
 '西安',
 '贵阳',
 '郑州',
 '重庆',
 '银川',
 '长春',
 '长沙',
 '香港']

In [41]:
# 获取温度信息
rege = r'-?(\d+)℃/-?(\d+)℃'
def trans_tem(tem):
    tmp_tem = re.match(rege, tem)
    mid_tem = (int(tmp_tem.group(1)) + int(tmp_tem.group(2)))/2
    return mid_tem


# 转换降水信息
def check_weather(wea):
    if wea[-1:] == '晴':
        wea = weather_dict['sun']
    elif wea[-1:] == '云':
        wea = weather_dict['cloud']
    elif wea[-1:] == '雨':
        wea = weather_dict['rain']
    elif wea[-1:] == '阴':
        wea = weather_dict['overcast']
    return wea

In [42]:

# 获取降水和温度信息
weather_data = map(check_weather, twoone['wea'].values.tolist())
weather_data = list(weather_data)
tem_data = map(trans_tem, twoone['tem'].values.tolist())
tem_data = list(tem_data)
weather_data_list = list(weather_data)
weather_data_list



[50,
 50,
 80,
 80,
 50,
 80,
 20,
 80,
 20,
 60,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 50,
 20,
 80,
 60,
 60,
 80,
 80,
 50,
 80,
 80,
 60,
 20,
 20,
 80,
 80,
 20,
 80]

In [43]:
# 降水和温度柱状图
pictorialbar = PictorialBar()
pictorialbar.add_xaxis(twoone_ctiy)
pictorialbar.add_yaxis(
    "降水量", weather_data,
    label_opts=opts.LabelOpts(is_show=False),
    symbol_size=18,
    symbol_repeat="fixed",
    symbol_offset=[0, 0],
    is_symbol_clip=True,
    itemstyle_opts=opts.ItemStyleOpts(color='#00BFFF'),
    symbol=SymbolType.ROUND_RECT
)

pictorialbar.add_yaxis(
    "温度", tem_data,
    label_opts=opts.LabelOpts(is_show=False),
    symbol_size=18,
    symbol_repeat="fixed",
    symbol_offset=[0, 0],
    is_symbol_clip=True,
#     itemstyle_opts=opts.ItemStyleOpts(color='yellow'),
    symbol=SymbolType.ARROW
)
# pictorialbar.reversal_axis()
pictorialbar.set_global_opts(
            legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(color="white")),
            title_opts=opts.TitleOpts(title="中秋节省会城市降雨和温度情况", title_textstyle_opts=opts.TextStyleOpts
                                      (color="white")),
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30, color="white"), is_show=True),
            yaxis_opts=opts.AxisOpts(
                axistick_opts=opts.AxisTickOpts(is_show=False),
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(opacity=0)
                ),
                axislabel_opts=opts.LabelOpts(formatter="{value}% 降水概率", color="white")
            ),
            datazoom_opts=opts.DataZoomOpts()
            
)

pictorialbar.render_notebook()
